<a href="https://colab.research.google.com/github/MuayThaiLegz/PracticeCrazy/blob/main/MLModelDescription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install --upgrade accelerate

!pip install PyPDF2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import PyPDF2
from pathlib import Path
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import nltk
from sklearn.model_selection import train_test_split

nltk.download('punkt')  # for sentence tokenization
nltk.download('stopwords')
# define a function for text cleaning
def clean_text(text):
    # convert all text to lowercase
    text = text.lower()
    # remove special characters and punctuation
    text = re.sub('[^a-z\s]', '', text)
    # remove stop words
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    text = ' '.join(filtered_words)
    # remove hyperlinks
    text = re.sub(r"http\S+", "", text)
    # remove numbers
    text = re.sub(r'\d+', '', text)
    # remove extra whitespaces
    text = re.sub('\s+', ' ', text).strip()
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, DataCollatorForLanguageModeling, TrainingArguments,TextDataset

In [4]:

#tokenizer = AutoTokenizer.from_pretrained("HamidRezaAttar/gpt2-product-description-generator")
#model = AutoModelForCausalLM.from_pretrained("HamidRezaAttar/gpt2-product-description-generator")
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')


In [5]:

def extractdata(filepath, pageNumTostart):
  with open(filepath, 'rb') as pdf_file:
      reader = PyPDF2.PdfReader(pdf_file)
      num_pages =  len(reader.pages)
      text = ""
      textList = []
      
      for page_num in range(num_pages)[pageNumTostart:]:
          page = reader.pages[page_num]
          textList.append(clean_text(page.extract_text()))

          booktextdf = pd.DataFrame(textList, columns=['text'])
          text += clean_text(page.extract_text())

  return text, booktextdf



In [6]:

filepath =  '/content/drive/MyDrive/textFiles/bernd_klein_python_and_machine_learning_letter.pdf'

text, booktextdf = extractdata(filepath, 23)        

In [7]:
booktextdf

,text
0,printiristargetshape bincount numpy counts num...
1,visualising features iris data set feauture da...
2,exercise look histograms features ie petal len...
3,exercise change xindex yindex script change xi...
4,axxysetxlabel xname axxysetylabel yname axxyle...
...,...
425,quantity suffixintraintest samples nprandommul...
426,numlabels automatically determined def evaluat...
427,trainsize numfeatures traindata shape xtfplace...
428,feed data model trainstep runfeeddict xbatchda...


In [8]:
dataminingpath  = '/content/drive/MyDrive/textFiles/Data Mining for Business Analytics by Galit Shmueli.pdf'

datatext, databooktextdf = extractdata(dataminingpath, 42)
        

In [9]:
databooktextdf

,text
0,chapter introduction business analytics busine...
1,static reports evolved userfriendly effective ...
2,uses predictive analytics widespread adoption ...
3,deploy capture hidden value must lurking somew...
4,term data mining means different things differ...
...,...
659,toys r us revenues data data reduction time se...
660,undersampling unequal importance classes unive...
661,variable selection variables categorical conti...
662,weight decay weighted average weighted samplin...


In [10]:
twobooks = pd.concat([booktextdf, databooktextdf], axis=0)

In [11]:
twobooks

,text
0,printiristargetshape bincount numpy counts num...
1,visualising features iris data set feauture da...
2,exercise look histograms features ie petal len...
3,exercise change xindex yindex script change xi...
4,axxysetxlabel xname axxysetylabel yname axxyle...
...,...
659,toys r us revenues data data reduction time se...
660,undersampling unequal importance classes unive...
661,variable selection variables categorical conti...
662,weight decay weighted average weighted samplin...


In [12]:
X_train, X_test= train_test_split(twobooks ,test_size=0.20, random_state=42)
train_path = 'train.txt'
test_path = 'test.txt'

X_train['text'].to_csv(train_path, index=False, header=False)
X_test['text'].to_csv(test_path, index=False, header=False)


In [13]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [14]:


training_args = TrainingArguments(
    output_dir="./headliner", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=2,  # batch size for evaluation
    eval_steps = 100, # Number of update steps between two evaluations.
    save_steps=50, # after # steps model is saved 
    warmup_steps=50,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    #prediction_loss_only=True,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,5.550200
1000,4.871600


TrainOutput(global_step=1450, training_loss=5.018326373922414, metrics={'train_runtime': 251.7588, 'train_samples_per_second': 23.018, 'train_steps_per_second': 5.759, 'total_flos': 189276833710080.0, 'train_loss': 5.018326373922414, 'epoch': 5.0})

In [26]:

from transformers import pipeline

headliner = pipeline('text-generation',
                model='headliner', 
                tokenizer=tokenizer,
                )
def get_mlinfo(headliner_pipeline, seed_text=None):
  return headliner_pipeline(seed_text)#[0]['generated_text'].split('\n')

In [28]:
get_mlinfo(headliner, seed_text="Deep learning")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Deep learningrate vectorizer import numpy asnp sklearnneighbors import numpy asnp psetinterpolatednormal neural networks\ntable training data time lmax min lmax max minmax inputvector vectorizer maximum iterations nparray'}]

In [17]:
trainer.save_model()

In [18]:
trainer.evaluate()


{'eval_loss': 4.966466426849365,
 'eval_runtime': 2.2868,
 'eval_samples_per_second': 128.125,
 'eval_steps_per_second': 64.281,
 'epoch': 5.0}

In [19]:
!pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.3 MB/s eta 0:00:00


In [20]:
"""from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')


# Open the PDF file in binary mode
with open('/content/drive/MyDrive/textFiles/Data Mining for Business Analytics by Galit Shmueli.pdf', 'rb') as pdf_file:
    reader = PyPDF2.PdfReader(pdf_file)
    num_pages =  len(reader.pages)
    text = ""
    
    # Read each page and extract text
    for page_num in range(num_pages)[42:]:
        page = reader.pages[page_num]
        text += page.extract_text()

# This will set the padding token to be the same as the EOS token (End of String)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(text.split(), truncation=True, padding=True, return_tensors='pt', max_length=512)
"""

'from transformers import GPT2LMHeadModel, GPT2Tokenizer\n\ntokenizer = GPT2Tokenizer.from_pretrained(\'distilgpt2\')\nmodel = GPT2LMHeadModel.from_pretrained(\'distilgpt2\')\n\n\n# Open the PDF file in binary mode\nwith open(\'/content/drive/MyDrive/textFiles/Data Mining for Business Analytics by Galit Shmueli.pdf\', \'rb\') as pdf_file:\n    reader = PyPDF2.PdfReader(pdf_file)\n    num_pages =  len(reader.pages)\n    text = ""\n    \n    # Read each page and extract text\n    for page_num in range(num_pages)[42:]:\n        page = reader.pages[page_num]\n        text += page.extract_text()\n\n# This will set the padding token to be the same as the EOS token (End of String)\ntokenizer.pad_token = tokenizer.eos_token\ninputs = tokenizer(text.split(), truncation=True, padding=True, return_tensors=\'pt\', max_length=512)\n'